In [1]:
# Loading Libraries
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm


C:\Users\pavan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\cuda\__init__.py:651: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ..\c10\cuda\CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [2]:

nlp = spacy.blank("en") # load a new spacy model
db = DocBin()


In [11]:

import json
annot = open('annotations.json')
valid = open('Validation_Annotations.json')
Data_annot = json.load(annot)
Valid_Annot = json.load(valid)

In [7]:
for text, annot in tqdm(Data_annot['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./data_annot_train.spacy")

100%|██████████| 1/1 [00:00<00:00,  8.49it/s]


In [12]:
for text, annot in tqdm(Valid_Annot['annotations']): 
    doc_valid = nlp.make_doc(text) 
    ents_valid = []
    for start, end, label in annot["entities"]:
        span = doc_valid.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents_valid.append(span)
    doc_valid.ents = ents_valid 
    db.add(doc_valid)

db.to_disk("./valid_annot_train.spacy")

100%|██████████| 1/1 [00:00<00:00, 38.95it/s]


In [8]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency


✘ The provided output file already exists. To force overwriting the config file,
set the --force or -F flag.



C:\Users\pavan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\cuda\__init__.py:651: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ..\c10\cuda\CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [13]:
! python -m spacy train config.cfg --output ./ --paths.train ./data_annot_train.spacy --paths.dev ./valid_annot_train.spacy

In [ ]:
custom_ner = spacy.load("/model-best") 

In [ ]:
text_doc = custom_ner('''

I. Parties of agreement

- Pavan Kumar with a mailing address of 404 Webb Ave, City of Toronto, Ontario, Canada (the "Borrower")
- CIBC Bank with a mailing address of 1 Jannet Avenue, City of Toronto, Ontario, Canada (the "Lender")

II. Amount
The loan amount given by lender to borrower is one hundred twenty thousand sixty-seven Dollars ($120,067.00) ("The Note")

III. Interest
The Note shall bear interest six percent (6%) compounded annually.

IV. Payment
The amount mentioned in this agreement (the "Note"), including the principal and any accrued interest, is

V. Payment Terms
Any delay in payment is subject to a fine with a flat amount of $50 for every week the payment is delayed.
All payments made by the Borrower shall be go into settling the the accrued interest  and any late fess and then into the payment of the principal amount.

VI. Prepayment
The borrower is able to pay back the Note in full at any time, thus terminating this agreement.
The borrower also can make additional payments at any time and this will take of from the amount of the latest installments. 

VII. Acceleration.
In case of Borrower's failure to pay any part of the principal or interest as and when due under this Note; or Borrower's becoming insolvent or not paying its debts as they become due. The lender has the right to declare an "Event of Acceleration" in which case the Lender has the right to to declare this Note immediately due and payable 

IX. Succession
This Note shall outlive the borrower and/or the lender in the even of their death. This note shall be binging to any of their successors.Date 10/18/2023


''')

In [ ]:
spacy.displacy.render(text_doc, style="ent", jupyter=True) # display in Jupyter

In [ ]:
for ent in text_doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
spacy.displacy.render(text_doc, style="ent", jupyter=True)